In [1]:
!pip install fasttext

In [3]:
import fasttext
train = '/kaggle/input/sst5-data/sst/sst_train.txt'
valid = '/kaggle/input/sst5-data/sst/sst_dev.txt'

In [33]:
hyper_params = {
    "lr": 0.5,
    "epoch": 100,
    "wordNgrams": 3,
    "dim": 100,
    "ws": 3,
}

In [34]:
model = fasttext.train_supervised(input=train, **hyper_params)
model.save_model('/kaggle/working/sst.bin')

In [8]:
import pandas as pd
# Read test data
df = pd.read_csv('/kaggle/input/sst5-data/sst/sst_test.txt', sep='\t', header=None,
                   names=['truth', 'sentence'],
                  )
df['truth'] = df['truth'].str.replace('__label__', '')
df['truth'] = df['truth'].astype(int).astype('category')
print(df.dtypes)
df.head()

truth       category
sentence      object
dtype: object


,truth,sentence
0,3,Effective but too-tepid biopic
1,4,If you sometimes like to go to the movies to h...
2,5,"Emerges as something rare , an issue movie tha..."
3,3,The film provides some great insight into the ...
4,5,Offers that rare combination of entertainment ...


In [15]:
len(df)

2210

In [9]:
def print_accuracy(df, pred_column):
    "Print f1 score and accuracy after making predictions"
    f1_macro = f1_score(df['truth'], df[pred_column], average='macro')
    acc = accuracy_score(df['truth'], df[pred_column])*100
    return f1_macro, acc

In [11]:
def fasttext_score(sentence):
    labels, probabilities = ft_model.predict(sentence, 1) # Predict just the top label, hence 1
    pred = int(labels[0][-1])
    return pred

In [13]:
from sklearn.metrics import f1_score, accuracy_score

In [35]:
ft_model = fasttext.load_model('/kaggle/working/sst.bin')
df['fasttext_pred'] = df['sentence'].str.lower().apply(fasttext_score)
# Get model accuracy and f1 score
acc = print_accuracy(df, 'fasttext_pred')
print("Macro F1-score: {}\nAccuracy: {}".format(acc[0], acc[1]))

Macro F1-score: 0.37271220120363047
Accuracy: 40.452488687782804
